# Analysis of treatment disparities for different trained binary predictors

#### Importing packages

In [3]:
import os
os.chdir('../')

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tueplots import bundles
from utils.notebook_helpers import *
import warnings
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
warnings.simplefilter('ignore')

In [5]:
pd.set_option('display.max_columns', 10)
pd.set_option("display.precision", 5)

In [6]:
bundles.aaai2024()

{'text.usetex': True,
 'font.family': 'serif',
 'text.latex.preamble': '\\usepackage{times} ',
 'figure.figsize': (3.3, 2.039512162874653),
 'figure.constrained_layout.use': True,
 'figure.autolayout': False,
 'savefig.bbox': 'tight',
 'savefig.pad_inches': 0.015,
 'font.size': 9,
 'axes.labelsize': 9,
 'legend.fontsize': 7,
 'xtick.labelsize': 7,
 'ytick.labelsize': 7,
 'axes.titlesize': 9}

In [7]:
plt.rcParams.update(bundles.aaai2024())
# Increase the resolution of all the plots below
plt.rcParams.update({"figure.dpi": 150})

In [8]:
root_path = '_models_trained/causal_nf'

## Getting different predictive decision-makers

In [9]:
import random
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import f1_score, accuracy_score
from fairlearn.metrics import MetricFrame, selection_rate, true_positive_rate, true_negative_rate, false_positive_rate, false_negative_rate, demographic_parity_difference, equalized_odds_difference, equal_opportunity_difference
from fairlearn.reductions import DemographicParity, ExponentiatedGradient, EqualizedOdds

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer

from fairlearn.postprocessing import ThresholdOptimizer, plot_threshold_optimizer
from fairlearn.reductions import DemographicParity, ExponentiatedGradient, GridSearch



In [14]:
def perform_ttd_dtd_analysis(data_df, mask, params, dname):
    treatment_col_idxs = range(params['dataset__num_sensitive'] + params['dataset__num_covariate'], 
                           params['dataset__num_sensitive'] + params['dataset__num_covariate'] + params['dataset__num_treatment'])
    total_features = params['dataset__num_sensitive'] + params['dataset__num_covariate'] + params['dataset__num_treatment'] + 1
    treatment_features = [c.split('_F')[0] for c in data_df.columns[treatment_col_idxs]]
    gender_col_name = data_df.columns[0].split('_F')[0]
    class_col_name = data_df.columns[total_features - 1].split('_F')[0]
    plot_params = {'rows': 1, 'cols': 3, 'figsize': (7, 1.7), 'plot_idx_legend': 1}
    df_sub = data_df[mask]
    result_dict = dtd_itd_analysis(df_sub, params, plot=False, plot_params=plot_params)

    #### TTD-DTD ####
    for k in list(result_dict.keys())[1:]:
        gender_val = int(k.split(gender_col_name + " ")[1])
        res = result_dict[k]
        cols = [f'{k}'] + [c for c in treatment_features]
        rows = []
        for meas in ['TD', 'DTD']:
            val_row = {cols[0]: meas}
            i = 1
            # Show treatment features
            for c in treatment_features:
                # val_row[cols[i]] = f'{res[meas][c][0]}+/-{res[meas][c][1]}'
                val_row[cols[i]] = f'{np.round(res[meas][c][0], 5)}'
                i += 1
            # Show outcome effects
            df_s = df_sub[df_sub[f'{gender_col_name}_F'] == gender_val]
            if meas == 'TD':
                cm = confusion_matrix(df_s[f'{class_col_name}_F'], df_s[f'{class_col_name}_TTD'], labels=[0, 1], normalize='true')
            elif meas == 'DTD':
                cm = confusion_matrix(df_s[f'{class_col_name}_F'], df_s[f'{class_col_name}_PSDT'], labels=[0, 1], normalize='true')
            cm_flat = cm.flatten()
            val_row['YF_0'], val_row['YF_1'] = cm_flat[1]*100, cm_flat[2]*100
            rows.append(val_row)
        df = pd.DataFrame(rows)
        print(df)
        print('-----------------------------------------------------------------')

    # LGD-ESI
    if dname == 'german':
        df_0 = df_sub[df_sub[f'{gender_col_name}_F'] == 0]
        df_1 = df_sub[df_sub[f'{gender_col_name}_F'] == 1]
        factual_female_lgd = np.mean((1 - df_0[f'{class_col_name}_F']) * df_0['credit_amount_F'])
        factual_male_lgd = np.mean((1 - df_1[f'{class_col_name}_F']) * df_1['credit_amount_F'])
        counterfactual_male_lgd = np.mean((1 - df_1[f'{class_col_name}_TTD']) * df_1['credit_amount_CF'])
        print(f"<LGD> Factual female: {factual_female_lgd}, Factual male: {factual_male_lgd}, Treated male: {counterfactual_male_lgd}")
        # print(f"<LGD> Factual female: {factual_female_lgd}, Factual male: {factual_male_lgd}")
        factual_female_esi = np.mean((df_0[f'{class_col_name}_F']) * (df_0['credit_amount_F'] * (df_0['duration_F']/12) / 100)) * 10
        factual_male_esi = np.mean((df_1[f'{class_col_name}_F']) * (df_1['credit_amount_F']  * (df_1['duration_F']/12) / 100)) * 10
        counterfactual_male_esi = np.mean((df_1[f'{class_col_name}_TTD']) * (df_1['credit_amount_CF'] * (df_1['duration_CF']/12) / 100)) * 10
        print(f"<ESI> Factual female: {factual_female_esi}, Factual male: {factual_male_esi}, Treated male: {counterfactual_male_esi}")
        # print(f"<ESI> Factual female: {factual_female_esi}, Factual male: {factual_male_esi}")
    elif dname == 'homecredit':
        df_0 = df_sub[df_sub[f'{gender_col_name}_F'] == 0]
        df_1 = df_sub[df_sub[f'{gender_col_name}_F'] == 1]
        factual_female_lgd = np.mean((1 - df_0[f'{class_col_name}_F']) * df_0['AMT_CREDIT_F'])
        factual_male_lgd = np.mean((1 - df_1[f'{class_col_name}_F']) * df_1['AMT_CREDIT_F'])
        counterfactual_male_lgd = np.mean((1 - df_1[f'{class_col_name}_TTD']) * df_1['AMT_CREDIT_CF'])
        # counterfactual_female_lgd = np.mean((1 - df_0[f'{class_col_name}_TTD']) * df_0['AMT_CREDIT_CF'])
        # print(f"<LGD> Factual female: {factual_female_lgd}, Treated female: {counterfactual_female_lgd},  Factual male: {factual_male_lgd}, Treated male: {counterfactual_male_lgd}")
        print(f"<LGD> Factual female: {factual_female_lgd}, Factual male: {factual_male_lgd}, Treated male: {counterfactual_male_lgd}")
        
        factual_female_esi = np.mean((df_0['AMT_ANNUITY_F'] * 12 * 15) - df_0['AMT_CREDIT_F'])
        factual_male_esi = np.mean((df_1['AMT_ANNUITY_F'] * 12 * 15) - df_1['AMT_CREDIT_F'])
        # counterfactual_female_esi =  np.mean((df_0[f'{class_col_name}_TTD']) * ((df_0['AMT_ANNUITY_CF'] * 12 * 15) -  df_0['AMT_CREDIT_CF'])) 
        counterfactual_male_esi = np.mean((df_1[f'{class_col_name}_TTD']) *  ((df_1['AMT_ANNUITY_CF'] * 12 * 15) -  df_1['AMT_CREDIT_CF'])) 
        # print(f"<ESI> Factual female: {factual_female_esi}, Treated Female: {counterfactual_female_esi},  Factual male: {factual_male_esi}, Treated male: {counterfactual_male_esi}")
        print(f"<ESI> Factual female: {factual_female_esi}, Factual male: {factual_male_esi}, Treated male: {counterfactual_male_esi}")

    

In [61]:
def new_predictor_based_analysis(data_name, clf_type, predictors, ftype='postprocess', root_path='_models_trained/causal_nf', plot_fair=False, do_train=False, train_on_cf=False):
    random.seed(42)
    np.random.seed(42)
    data_df_all = load_csv_file(root_path, data_name, 'all')
    params = load_yaml_config(root_path, data_name)
    total_features = params['dataset__num_sensitive'] + params['dataset__num_covariate'] + params['dataset__num_treatment'] + 1
    
    from sklearn.model_selection import train_test_split
    data_df_train, data_df_test = train_test_split(data_df_all, test_size=0.4, shuffle=True, random_state=12345, stratify=data_df_all[data_df_all.columns[total_features-1]])
    # data_df_train = load_csv_file(root_path, data_name, 'train')
    # data_df_test = load_csv_file(root_path, data_name, 'test')
    np_train, np_test = data_df_train.to_numpy(), data_df_test.to_numpy()

    S_train, X_train, Y_train = np_train[:, 0], np_train[:, 1:total_features-1], np_train[:, total_features-1]
    S_test, X_test, Y_test = np_test[:, 0], np_test[:, 1:total_features-1], np_test[:, total_features-1]
    
    if data_name in ['german', 'homecredit'] and train_on_cf:
        # print(X_train.shape[1], params['dataset__num_covariate'])
        print(list(data_df_all.columns)[total_features+params['dataset__num_sensitive']+params['dataset__num_covariate']:2*total_features-1])
        X_train[S_train == 1,params['dataset__num_sensitive']-1+params['dataset__num_covariate']:] = np_train[S_train == 1, total_features+params['dataset__num_sensitive']+params['dataset__num_covariate']:2*total_features-1]
        X_test[S_test == 1,params['dataset__num_sensitive']-1+params['dataset__num_covariate']:] = np_test[S_test == 1, total_features+params['dataset__num_sensitive']+params['dataset__num_covariate']:2*total_features-1]
        print(list(data_df_all.columns)[2*total_features])
        Y_train[S_train == 1], Y_test[S_test == 1] = np_train[S_train == 1, 2*total_features], np_test[S_test == 1, 2*total_features]
    if data_name == 'homecredit':
        Y_train, Y_test = 1.0 - Y_train, 1.0 - Y_test

    # print(data_df_train.columns[0], data_df_train.columns[1:total_features-1], data_df_train.columns[total_features-1])
    print("Y-test-0:", np.sum(Y_test==0), "Y-test-1:", np.sum(Y_test==1))    
    ohe_indices = params['dataset__categorical_dims']
    ohe_indices.pop()
    ohe_indices.pop(0)
    ohe_indices = [i-1 for i in ohe_indices]
    all_indices = list(range(X_train.shape[1]))
    scale_indices = [i for i in all_indices if i not in ohe_indices]

    cols = [c for c in data_df_train.columns if c.endswith("_F")]
    
    # print(ohe_indices, scale_indices, X_train.shape, total_features, len(cols))
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('ohe', OneHotEncoder(), ohe_indices),
            ('scale', StandardScaler(), scale_indices)
        ]
    )

    if clf_type == 'lr': clf = LogisticRegression(class_weight='balanced')
    else: raise NotImplementedError

    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", clf),
        ]
    )
    for fairness in predictors:
        if fairness == 'none':
            mask = np.ones(data_df_test.shape[0], dtype=bool)
            if not do_train:
                print(f"*** FULL TEST {data_df_test.shape[0]} ***")
                perform_ttd_dtd_analysis(data_df_test, mask, params, data_name)
            else:
                print(f"*** FULL TRAIN {data_df_train.shape[0]} ***")
                mask = np.ones(data_df_train.shape[0], dtype=bool)
                perform_ttd_dtd_analysis(data_df_train, mask, params, data_name)
            continue
        if fairness == 'accuracy':
            pipeline.fit(X_train, Y_train)
            unmitigated_test_preds = pipeline.predict(X_test)
            unmitigated_train_preds = pipeline.predict(X_train)
            unmitigated_mask = unmitigated_test_preds == 1.0
            mask_ = unmitigated_train_preds == 1.0
            if not do_train:
                print(f"*** PREDICTOR-{fairness} TEST {np.sum(unmitigated_mask)}***")
            else:
                print(f"*** PREDICTOR-{fairness} TRAIN {np.sum(mask_)}***")
            print("TEST Accuracy:", accuracy_score(Y_test, unmitigated_test_preds), "F1:", f1_score(Y_test, unmitigated_test_preds),
                "DPD:", demographic_parity_difference(Y_test, unmitigated_test_preds, sensitive_features=S_test),
                 "EOD:", equalized_odds_difference(Y_test, unmitigated_test_preds, sensitive_features=S_test),
                 "EOP:", equal_opportunity_difference(Y_test, unmitigated_test_preds, sensitive_features=S_test))
            print("TRAIN Accuracy:", accuracy_score(Y_train, unmitigated_train_preds), "F1:", f1_score(Y_train, unmitigated_train_preds),
                "DPD:", demographic_parity_difference(Y_train, unmitigated_train_preds, sensitive_features=S_train),
                 "EOD:", equalized_odds_difference(Y_train, unmitigated_train_preds, sensitive_features=S_train),
                 "EOP:", equal_opportunity_difference(Y_train, unmitigated_train_preds, sensitive_features=S_train))
            # print(np.sum(Y_test), np.sum(unmitigated_mask))
            if not do_train:
                perform_ttd_dtd_analysis(data_df_test, unmitigated_mask, params, data_name)
            else:
                perform_ttd_dtd_analysis(data_df_train, mask_, params, data_name)
            continue
        # else:
        if ftype == 'threshold':
            pipeline.fit(X_train, Y_train)
            threshold_optimizer = ThresholdOptimizer(
                estimator=pipeline,
                constraints=fairness,
                objective='balanced_accuracy_score',
                # flip=True,
                # predict_method="predict_proba",
                # prefit=False,
                predict_method='predict_proba',
                prefit=True,
                grid_size=5000,
            )
            ##### Learn and apply threshold change on test??? #####
            threshold_optimizer.fit(X_train, Y_train, sensitive_features=S_train)
            ########################
            train_preds = threshold_optimizer.predict(X_train, sensitive_features=S_train, random_state=42)
            test_preds = threshold_optimizer.predict(X_test, sensitive_features=S_test, random_state=42)
            if plot_fair:
                plot_threshold_optimizer(threshold_optimizer)
        else:
            if fairness == 'demographic_parity':
                constraint = DemographicParity()
            elif fairness == 'equalized_odds':
                constraint = EqualizedOdds()
            if ftype == 'exp_gradient':
                method = ExponentiatedGradient(
                    estimator=pipeline,
                    constraints=constraint,
                    sample_weight_name="classifier__sample_weight",
                )
                method.fit(X_train, Y_train, sensitive_features=S_train)
                test_preds = method.predict(X_test, random_state=42)
                train_preds = method.predict(X_train, random_state=42)
                mask = test_preds == 1.0
            elif ftype == 'grid_search':
                method = GridSearch(
                    estimator=pipeline,
                    constraints=constraint,
                    sample_weight_name="classifier__sample_weight",
                    grid_limit=5,
                    grid_size=20,
                    constraint_weight=0.1,
                )
                method.fit(X_train, Y_train, sensitive_features=S_train)
                train_preds = method.predict(X_train)
                test_preds = method.predict(X_test)
        mask = test_preds == 1.0
        mask_ = train_preds == 1.0
        if not do_train:
            print(f"*** PREDICTOR-{fairness} TEST {np.sum(mask)}***")
        else:
            print(f"*** PREDICTOR-{fairness} TRAIN {np.sum(mask_)}***")
        print("TEST Accuracy:", accuracy_score(Y_test, test_preds), "F1:", f1_score(Y_test, test_preds),
            "DPD:", demographic_parity_difference(Y_test, test_preds, sensitive_features=S_test),
             "EOD:", equalized_odds_difference(Y_test, test_preds, sensitive_features=S_test),
             "EOP:", equal_opportunity_difference(Y_test, test_preds, sensitive_features=S_test))

        print("TRAIN Accuracy:", accuracy_score(Y_train, train_preds), "F1:", f1_score(Y_train, train_preds),
                "DPD:", demographic_parity_difference(Y_train, train_preds, sensitive_features=S_train),
                 "EOD:", equalized_odds_difference(Y_train, train_preds, sensitive_features=S_train),
                 "EOP:", equal_opportunity_difference(Y_train, train_preds, sensitive_features=S_train))
        
        if not do_train:
            perform_ttd_dtd_analysis(data_df_test, mask, params, data_name)
        else:
            mask_ = train_preds == 1.0
            perform_ttd_dtd_analysis(data_df_train, mask_, params, data_name)
    

---

In [68]:
predictors = ('none', 'accuracy', 'demographic_parity', 'equalized_odds')
# predictors = ('accuracy', 'true_positive_rate_parity',)
new_predictor_based_analysis('german', 'lr', predictors, ftype='threshold', plot_fair=False, do_train=False)

Y-test-0: 120 Y-test-1: 280
*** FULL TEST 400 ***
  gender 0 duration credit_amount installment_commitment     YF_0     YF_1
0       TD  1.14706       272.147                0.00092  4.25532  2.70270
1      DTD  0.42426      133.3877                0.00088  4.25532  1.35135
-----------------------------------------------------------------
  gender 1  duration credit_amount installment_commitment      YF_0     YF_1
0       TD  -0.86627     -237.9707               -0.00075  12.32877  0.97087
1      DTD  -0.57887      -189.677               -0.00088  12.32877  0.48544
-----------------------------------------------------------------
<LGD> Factual female: 1510.1818181818182, Factual male: 1063.326164874552, Treated male: 756.8132703225806
<ESI> Factual female: 300.930303030303, Factual male: 461.51705495818396, Treated male: 391.88985933673314
*** PREDICTOR-accuracy TEST 238***
TEST Accuracy: 0.7 F1: 0.7683397683397685 DPD: 0.05918421754198877 EOD: 0.1844943165257942 EOP: 0.080949881920755

In [15]:
predictors = ('none', 'accuracy', 'demographic_parity', 'equalized_odds')
new_predictor_based_analysis('homecredit', 'lr', predictors, ftype='threshold', plot_fair=False, do_train=False)

Y-test-0: 2983 Y-test-1: 36560
*** FULL TEST 39543 ***
  CODE_GENDER 0 AMT_CREDIT AMT_ANNUITY     YF_0     YF_1
0            TD    7834.47    1005.262  0.24677  0.22185
1           DTD     -797.4     708.244  0.07686  0.22185
-----------------------------------------------------------------
  CODE_GENDER 1 AMT_CREDIT AMT_ANNUITY     YF_0     YF_1
0            TD   -6778.88     -957.37  0.05067  2.45763
1           DTD     857.12    -604.941  0.07601  1.35593
-----------------------------------------------------------------
<LGD> Factual female: 585921.0245456602, Factual male: 578769.165079487, Treated male: 570638.6737408034
<ESI> Factual female: 4385747.622935676, Factual male: 4694030.738038553, Treated male: 387957.06315306044
*** PREDICTOR-accuracy TEST 26258***
TEST Accuracy: 0.6917785701641251 F1: 0.805979177942628 DPD: 0.13642192887660476 EOD: 0.13102568879611787 EOP: 0.13102568879611787
TRAIN Accuracy: 0.695147857166942 F1: 0.8083884367582231 DPD: 0.13200101463015512 EOD: 0.12

In [17]:
predictors = ('none', 'accuracy', 'demographic_parity', 'equalized_odds')
new_predictor_based_analysis('hmdaTX', 'lr', predictors, ftype='threshold', plot_fair=False, do_train=False)

Y-test-0: 8961 Y-test-1: 150781
*** FULL TEST 159742 ***
  applicant_sex 0 loan_amount_000s preapproval     YF_0     YF_1
0              TD          19.3819         0.0  1.60156  0.28663
1             DTD          2.70135         0.0  1.64062  0.13749
-----------------------------------------------------------------
  applicant_sex 1 loan_amount_000s preapproval     YF_0     YF_1
0              TD        -19.81134         0.0  3.01515  0.16965
1             DTD         -3.64966         0.0  2.37463  0.10105
-----------------------------------------------------------------
*** PREDICTOR-accuracy TEST 114723***
TEST Accuracy: 0.7741983949117953 F1: 0.8641451729540798 DPD: 0.012707225380230724 EOD: 0.013713086813817554 EOP: 0.013713086813817554
TRAIN Accuracy: 0.7733085154332838 F1: 0.863535642325607 DPD: 0.010846548171854153 EOD: 0.012331993618532988 EOP: 0.012331993618532988
  applicant_sex 0 loan_amount_000s preapproval  YF_0     YF_1
0              TD         20.98283         0.0   0.

---

## Training predictors on fair data

In [62]:
predictors = ('accuracy', 'demographic_parity', 'equalized_odds')
new_predictor_based_analysis('homecredit', 'lr', predictors, ftype='threshold', plot_fair=False, do_train=False, train_on_cf=True)

['AMT_CREDIT_CF', 'AMT_ANNUITY_CF']
TARGET_TTD
Y-test-0: 2960 Y-test-1: 36583
*** PREDICTOR-accuracy TEST 26240***
TEST Accuracy: 0.6914498141263941 F1: 0.8057876892221002 DPD: 0.12612494763973214 EOD: 0.12118181727417243 EOP: 0.12118181727417243
TRAIN Accuracy: 0.6943048858616853 F1: 0.8078463788389395 DPD: 0.12168168911349941 EOD: 0.11422963216363125 EOP: 0.11422963216363125
  CODE_GENDER 0 AMT_CREDIT AMT_ANNUITY     YF_0     YF_1
0            TD    8475.28     1018.67  0.13279  0.31898
1           DTD     -945.8     683.738  0.04980  0.00000
-----------------------------------------------------------------
  CODE_GENDER 1 AMT_CREDIT AMT_ANNUITY     YF_0     YF_1
0            TD        0.0         0.0  0.00000  0.00000
1           DTD    10226.5     275.594  0.09673  0.66225
-----------------------------------------------------------------
<LGD> Factual female: 630534.2520988182, Factual male: 641463.720967237, Treated male: 641463.720967237
<ESI> Factual female: 4444490.554515801, F

In [69]:
predictors = ('accuracy', 'demographic_parity', 'equalized_odds')
# predictors = ('true_positive_rate_parity',)
new_predictor_based_analysis('german', 'lr', predictors, ftype='threshold', plot_fair=False, do_train=False, train_on_cf=True)

['duration_CF', 'credit_amount_CF', 'installment_commitment_CF']
class_TTD
Y-test-0: 113 Y-test-1: 287
*** PREDICTOR-accuracy TEST 240***
TEST Accuracy: 0.6925 F1: 0.7666034155597723 DPD: 0.042655291922154115 EOD: 0.21147646679561574 EOP: 0.1259357949498794
TRAIN Accuracy: 0.7716666666666666 F1: 0.8245838668373879 DPD: 0.04456802996948983 EOD: 0.030112789955309638 EOP: 0.030112789955309638
  gender 0 duration credit_amount installment_commitment  YF_0     YF_1
0       TD  0.87355      189.8925                0.00115   0.0  3.38983
1      DTD  0.35855      116.7399                0.00124   0.0  1.69492
-----------------------------------------------------------------
  gender 1 duration credit_amount installment_commitment     YF_0    YF_1
0       TD      0.0           0.0                    0.0  0.00000  0.0000
1      DTD  0.15253         4.883                0.00015  3.57143  1.3986
-----------------------------------------------------------------
<LGD> Factual female: 282.36231884057